In [1]:
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import __version__ as tf_version, float32 as tf_float32, Variable
from tensorflow.keras import Sequential, Model
from tensorflow.keras.backend import variable, dot as k_dot, sigmoid, relu
from tensorflow.keras.layers import Dense, Input, Concatenate, Layer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.random import set_seed as tf_set_seed
from numpy import __version__ as np_version, unique, array, mean, argmax
from numpy.random import seed as np_seed, choice
from pandas import __version__ as pd_version, read_csv, DataFrame, concat
from sklearn import __version__ as sk_version
from sklearn.preprocessing import normalize
print("tensorflow version:", tf_version)
print("numpy version:", np_version)
print("pandas version:", pd_version)
print("scikit-learn version:", sk_version)

tensorflow version: 2.10.0
numpy version: 1.20.3
pandas version: 1.3.4
scikit-learn version: 0.24.2


In [2]:
X_train,Y_train = np.load("X_train.npy"),np.load("Y_train.npy")
X_test,Y_test = np.load("X_test.npy"),np.load("Y_test.npy")

In [3]:
X_train.shape,X_test.shape

((66757, 2, 5, 49), (16690, 2, 5, 49))

In [4]:
#[blue_top,red_top,blue_jgl,red_jgl,blue_mid,red_mid,blue_adc,red_adc,blue_sup,red_sup]
def cut_down_data(X):
    blue_top,red_top,blue_jgl,red_jgl,blue_mid,red_mid,blue_adc,red_adc,blue_sup,red_sup = [],[],[],[],[],[],[],[],[],[]
    for x in X:
        blue_top.append(x[0][0])
        red_top.append(x[1][0])
        blue_jgl.append(x[0][1])
        red_jgl.append(x[1][1])
        blue_mid.append(x[0][2])
        red_mid.append(x[1][2])
        blue_adc.append(x[0][3])
        red_adc.append(x[1][3])
        blue_sup.append(x[0][4])
        red_sup.append(x[1][4])
    return(np.array(blue_top),np.array(red_top),np.array(blue_jgl),np.array(red_jgl),np.array(blue_mid),np.array(red_mid),np.array(blue_adc),np.array(red_adc),np.array(blue_sup),np.array(red_sup))

In [5]:
Train_blue_top,Train_red_top,Train_blue_jgl,Train_red_jgl,Train_blue_mid,Train_red_mid,Train_blue_adc,Train_red_adc,Train_blue_sup,Train_red_sup = cut_down_data(X_train)
Test_blue_top,Test_red_top,Test_blue_jgl,Test_red_jgl,Test_blue_mid,Test_red_mid,Test_blue_adc,Test_red_adc,Test_blue_sup,Test_red_sup = cut_down_data(X_test)

In [11]:
Train_data = [Train_blue_top,Train_red_top,Train_blue_jgl,Train_red_jgl,Train_blue_mid,Train_red_mid,Train_blue_adc,Train_red_adc,Train_blue_sup,Train_red_sup]
Test_data = [Test_blue_top,Test_red_top,Test_blue_jgl,Test_red_jgl,Test_blue_mid,Test_red_mid,Test_blue_adc,Test_red_adc,Test_blue_sup,Test_red_sup]

In [7]:
Test_red_top.shape

(16690, 49)

In [8]:
_matchup_shape = 2 * X_train.shape[-1]
_champion_shape = X_train.shape[-1]

In [9]:
tf_set_seed(1102)
np_seed(1102)

blue_top_input = Input(shape = _champion_shape, name = "Blue_top_input")
red_top_input = Input(shape = _champion_shape, name = "Red_top_input")
top_input = Concatenate(name = "Merged_top_input")([blue_top_input,red_top_input])
top_matchup = Dense(32, activation = 'relu', name="Top_matchup")(top_input)

blue_jgl_input = Input(shape = _champion_shape, name = "Blue_jgl_input")
red_jgl_input = Input(shape = _champion_shape, name = "Red_jgl_input")
jgl_input = Concatenate(name = "Merged_jgl_input")([blue_jgl_input,red_jgl_input])
jgl_matchup = Dense(32, activation = 'relu',name = 'Jgl_matchup')(jgl_input) 

blue_jgl_top_input = Concatenate(name = 'Blue_jgl_top_input')([blue_top_input,blue_jgl_input])
blue_jgl_top = Dense(16,activation='relu', name = 'Blue_jgl_top')(blue_jgl_top_input)
red_jgl_top_input = Concatenate(name = 'Red_jgl_top_input')([red_top_input,red_jgl_input])
red_jgl_top = Dense(16,activation='relu', name = 'Red_jgl_top')(red_jgl_top_input)

jgl_top_input = Concatenate(name = "jgl_top_matchup_input")([blue_jgl_top,red_jgl_top])
jgl_top_matchup = Dense(8, name = 'jgl_top_matchup')(jgl_top_input) 


blue_mid_input = Input(shape = _champion_shape, name = "Blue_mid_input")
red_mid_input = Input(shape = _champion_shape,name = 'Red_mid_input')
mid_input = Concatenate(name='Merged_mid_input')([blue_mid_input,red_mid_input])
mid_matchup = Dense(32, activation = 'relu',name = "Mid_matchup")(mid_input)

blue_jgl_mid_input = Concatenate(name = 'Blue_jgl_mid_input')([blue_mid_input,blue_jgl_input])
blue_jgl_mid = Dense(16,activation='relu',name='Blue_jgl_mid')(blue_jgl_mid_input)
red_jgl_mid_input = Concatenate(name='Red_jgl_mid_input')([red_mid_input,red_jgl_input])
red_jgl_mid = Dense(16,activation='relu',name='Red_jgl_mid')(red_jgl_mid_input)

jgl_mid_input = Concatenate(name='Jgl_mid_matchup_input')([blue_jgl_mid,red_jgl_mid])
jgl_mid_matchup = Dense(8,name='Jgl_mid_matchup')(jgl_mid_input) 



blue_adc_input = Input(shape = _champion_shape,name='Blue_adc_input')
red_adc_input = Input(shape = _champion_shape,name='Red_adc_input')
adc_input = Concatenate(name='Merged_adc_input')([blue_adc_input,red_adc_input])
adc_matchup = Dense(32, activation = 'relu',name='ADC_matchup')(adc_input)

blue_sup_input = Input(shape = _champion_shape,name='Blue_sup_input')
red_sup_input = Input(shape = _champion_shape,name='Red_sup_input')
sup_input = Concatenate(name='Sup_matchup_input')([blue_sup_input,red_sup_input])
sup_matchup = Dense(32, activation = 'relu',name='Sup_matchup')(sup_input)

botlane_input = Concatenate(name='Botlane_matchup_input')([adc_matchup,sup_matchup])
botlane_matchup = Dense(16,activation='relu',name='Botlane_matchup')(botlane_input)

blue_jgl_sup_input = Concatenate(name='Blue_jgl_sup_input')([blue_sup_input,blue_jgl_input])
blue_jgl_sup = Dense(16,activation='relu',name='Blue_jgl_sup')(blue_jgl_sup_input)
red_jgl_sup_input = Concatenate(name='Red_jgl_sup_input')([red_sup_input,red_jgl_input])
red_jgl_sup = Dense(16,activation='relu',name='Red_jgl_sup')(red_jgl_sup_input)

jgl_sup_input = Concatenate(name='Merged_jgl_sup_input')([blue_jgl_sup,red_jgl_sup])
jgl_sup_matchup = Dense(8,name = 'Merged_jgl_sup')(jgl_sup_input) 


all_input = Concatenate(name = "Merged_all_input")([top_matchup,jgl_top_matchup,mid_matchup,jgl_mid_matchup,botlane_matchup,jgl_sup_matchup])
dense_layer = Dense(32,activation='tanh',name='Merged_all')(all_input)

final_layer = Dense(1,activation='sigmoid',name='Output')(dense_layer)


BDC = Model(inputs=[blue_top_input,red_top_input,blue_jgl_input,red_jgl_input,blue_mid_input,red_mid_input,blue_adc_input,red_adc_input,blue_sup_input,red_sup_input],outputs = final_layer, name="BDC")
BDC.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
BDC.summary()

Model: "BDC"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Blue_top_input (InputLayer)    [(None, 49)]         0           []                               
                                                                                                  
 Red_top_input (InputLayer)     [(None, 49)]         0           []                               
                                                                                                  
 Blue_jgl_input (InputLayer)    [(None, 49)]         0           []                               
                                                                                                  
 Red_jgl_input (InputLayer)     [(None, 49)]         0           []                               
                                                                                                

 jgl_top_matchup (Dense)        (None, 8)            264         ['jgl_top_matchup_input[0][0]']  
                                                                                                  
 Mid_matchup (Dense)            (None, 32)           3168        ['Merged_mid_input[0][0]']       
                                                                                                  
 Jgl_mid_matchup (Dense)        (None, 8)            264         ['Jgl_mid_matchup_input[0][0]']  
                                                                                                  
 Botlane_matchup (Dense)        (None, 16)           1040        ['Botlane_matchup_input[0][0]']  
                                                                                                  
 Merged_jgl_sup (Dense)         (None, 8)            264         ['Merged_jgl_sup_input[0][0]']   
                                                                                                  
 Merged_al

In [10]:
BDC.fit(Train_data,Y_train, batch_size=64,epochs=100,validation_split=0.2)

Epoch 1/100


C:\Users\antoi\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


835/835 [==============================] - 24s 28ms/step - loss: 0.6968 - accuracy: 0.5010 - val_loss: 0.6934 - val_accuracy: 0.4979
Epoch 2/100
835/835 [==============================] - 24s 28ms/step - loss: 0.6934 - accuracy: 0.5071 - val_loss: 0.6943 - val_accuracy: 0.5067
Epoch 3/100
835/835 [==============================] - 24s 29ms/step - loss: 0.6926 - accuracy: 0.5114 - val_loss: 0.6935 - val_accuracy: 0.5039
Epoch 4/100
835/835 [==============================] - 24s 29ms/step - loss: 0.6920 - accuracy: 0.5191 - val_loss: 0.6954 - val_accuracy: 0.5007
Epoch 5/100
835/835 [==============================] - 24s 29ms/step - loss: 0.6913 - accuracy: 0.5250 - val_loss: 0.6941 - val_accuracy: 0.5164
Epoch 6/100
835/835 [==============================] - 23s 28ms/step - loss: 0.6900 - accuracy: 0.5295 - val_loss: 0.6951 - val_accuracy: 0.5116
Epoch 7/100
835/835 [==============================] - 24s 29ms/step - loss: 0.6887 - accuracy: 0.5354 - val_loss: 0.6929 - val_accuracy: 0.51

835/835 [==============================] - 22s 26ms/step - loss: 0.5164 - accuracy: 0.7337 - val_loss: 0.8510 - val_accuracy: 0.5413
Epoch 58/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5149 - accuracy: 0.7353 - val_loss: 0.8646 - val_accuracy: 0.5386
Epoch 59/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5119 - accuracy: 0.7357 - val_loss: 0.8674 - val_accuracy: 0.5353
Epoch 60/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5089 - accuracy: 0.7386 - val_loss: 0.8688 - val_accuracy: 0.5417
Epoch 61/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5084 - accuracy: 0.7407 - val_loss: 0.8730 - val_accuracy: 0.5404
Epoch 62/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5061 - accuracy: 0.7413 - val_loss: 0.8733 - val_accuracy: 0.5450
Epoch 63/100
835/835 [==============================] - 22s 26ms/step - loss: 0.5047 - accuracy: 0.7420 - val_loss: 0.8813 - val_accuracy

In [20]:
X_train.shape

(66757, 2, 5, 49)

In [13]:
res = BDC.predict(Test_data)

522/522 [==============================] - 4s 8ms/step


In [17]:
aux = []
for i in range(len(res)):
    if res[i]>0.5:
        aux.append(1)
    else:
        aux.append(0)

In [18]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [19]:
accuracy_score(Y_test,aux)

0.5437387657279809

In [21]:
confusion_matrix(Y_test,aux)

array([[4410, 4037],
       [3578, 4665]], dtype=int64)